# Elecciones presidenciales en Twitter

Actualmente la gran parte del discurso publico se lleva a cabo en redes sociales. Por un lado esto permite a cualquier persona informarse y expresarse de manera instantánea y directa con el resto del mundo. Por el otro, permite hacer uso de tecnologías para amplificar mensajes o ideas artificialmente. 

El objetivo de este estudio es cuantificar la presencia de Bots, o cuentas automatizadas, en el discurso publico en redes sociales en el contexto de las elecciones presidenciales en Chile del año 2021.

La metodología consiste en descargar todos los mensajes que mencionen a uno de los candidatos presidenciales y analizar si los usuarios que generaron el mensaje son personas reales o cuentas automatizadas, además de a qué candidato apoyan.

## Data

In [14]:
import numpy as np
import pandas as pd

tweets_df = pd.read_json("tweets/all_tweets.json")
users_df = pd.read_json("data/clean/users.json").drop("english", axis=1)

print("Primer Tweet del estudio:", tweets_df.created_at.min())
print("Ultimo Tweet del estudio:", tweets_df.created_at.max())
print("Duracion:", pd.to_timedelta(tweets_df.created_at.max() - tweets_df.created_at.min(), unit='D'))
print("Tweets:", len(tweets_df))
print("Usuarios:", len(tweets_df.author_id.unique()))

Primer Tweet del estudio: 2021-11-26 06:59:07
Ultimo Tweet del estudio: 2021-12-03 11:49:36
Duracion: 7 days 04:50:29
Tweets: 2341610
Usuarios: 172135


El estudio se realizo exclusivamente en Twitter por la facilidad que otorga para entregar datos. El periodo consiste de la semana entre el 26.11.2021y el 03.12.2021 y los criterios de búsqueda son únicamente que mencionen las palabras **Kast** o **Boric** y que el texto del mensaje sea en español.

Esto se traduce en **2.341.610** tweets generados por **172.135** usuarios.

## Hashtag mas populares

Primero se ordenaron los Tweets según hashtags.  Los hashtags tiene como fin clasificar un mensaje en una categoría designada por el usuario que lo creó. Distinguir si un mensaje apoya a uno u otro candidato no es una tarea fácil, por lo que decidimos usar los hashtags como proxy para determinar a quién apoya cada tweet.

Los hashtags mas populares son:

In [15]:
def top_hashtags(df):
    hashtags_dict = {}
    for index, values in df.entities.items():
        if values and ('hashtags' in values):
            hashtags = values['hashtags']       
            for hashtag in hashtags:
                # check if hashtags is in the dictionary
                tag = "#" + hashtag['tag']
                if ("boric" in tag.lower()) or ("kast" in tag.lower()):
                    if tag in hashtags_dict:
                        hashtags_dict[tag] += 1
                    else:
                        hashtags_dict[tag] = 1
    hashtags_df = pd.DataFrame.from_dict(hashtags_dict, orient='index', columns=['count'])
    return hashtags_df.sort_values(by='count', ascending=False)

In [16]:
top_hashtags_df = top_hashtags(tweets_df)
top_hashtags_df.head(20)

,count
#Boric,45269
#BoricPresidente,35257
#Kast,31339
#MujeresPorKast,21976
#TodoChileVotaKast,19090
#KastPresidente2022,17759
#TodosKast,12656
#BoricPresidente2022,7749
#BoricNoSeAtreveDebatir,7065
#kast,6617


Para nuestro analisis excluiremos los hashtags #Kast, #Boric , #kast y #boric. Esto dado que tanto un comentario positivo como negativo al respecto de los candidatos puede incluirlos y es muy dificil clasificar a quien apoyan. 

La lista de los 10 hashtags mas populares excluyendo los hastags mencionados se ve asi:

In [17]:
hashtags_list = ['#BoricPresidente',
 '#MujeresPorKast',
 '#TodoChileVotaKast',
 '#KastPresidente2022',
 '#TodosKast',
 '#BoricPresidente2022',
 '#BoricNoSeAtreveDebatir',
 '#BoricNoSeLaPuede',
 '#BoricMiente',
 '#KastPresidente']

top_hashtags_df = top_hashtags_df[top_hashtags_df.index.isin(hashtags_list)]
top_hashtags_df

,count
#BoricPresidente,35257
#MujeresPorKast,21976
#TodoChileVotaKast,19090
#KastPresidente2022,17759
#TodosKast,12656
#BoricPresidente2022,7749
#BoricNoSeAtreveDebatir,7065
#BoricNoSeLaPuede,5610
#BoricMiente,4733
#KastPresidente,4725


A primera vista podemos observar que hay hashtags que apoyan directamente a un candidato y otros que se manifiestan contrarios a un candidato. 

In [18]:
hashtags_support_dict = {
 "#BoricPresidente": "Boric",
 "#MujeresPorKast": "Kast",
 "#TodoChileVotaKast": "Kast",
 "#KastPresidente2022": "Kast",
 "#TodosKast": "Kast",
 "#BoricPresidente2022": "Boric",
 "#BoricNoSeAtreveDebatir": "Kast",
 "#BoricNoSeLaPuede": "Kast",
 "#BoricMiente": "Kast",
 "#KastPresidente": "Kast"
 }

**De aqui en adelante solo trabajaremos con los tweets pertenecientos a los hashtags anteriormente mencionados**

In [19]:
# asignar cuentas a candidatos
def candidate_support(text):
    for key in hashtags_support_dict:
        if key in text:
            support =  hashtags_support_dict[key]
    if "support" not in locals():
        support = np.nan
    return support 

In [20]:
tweets_df["candidate_support"] = tweets_df.text.apply(lambda x: candidate_support(x))
tweets_df = tweets_df[tweets_df.candidate_support.notna()]
total_hashtags_users = len(tweets_df.author_id.unique())

print("Total de usuarios de los top 10 hashtags:", total_hashtags_users)

Total de usuarios de los top 10 hashtags: 30572


## Cuentas eliminadas

Antes de ver el resultado el análisis es importante destacar un hallazgo curioso: el **26,26%** del total de las cuentas analizadas en el muestreo se encuentran eliminadas. Una cuenta puede suspenderse por tres razones:

- El usuario voluntariamente cierra su cuenta
- La cuenta se encuentra inactiva por un largo periodo de tiempo
- Twitter suspende la cuenta por [violación al código de conducta](https://help.twitter.com/en/rules-and-policies/twitter-rules)

In [21]:
users_df = users_df[users_df.id_str.isin(tweets_df.author_id.unique())]
tweets_df = tweets_df.merge(users_df, how="left", left_on="author_id", right_on="id_str")

usuarios_activos = len(users_df[users_df.status == "active"])
usuarios_eliminados = len(users_df[users_df.status == "deleted"])
ratio_borrados = usuarios_eliminados / (usuarios_eliminados + usuarios_activos)

print("Total de usuarios:", total_hashtags_users)
print("Usuarios activos:", usuarios_activos)
print("Usuarios eliminados:", usuarios_eliminados)
print(f"Proporcion de usuarios borrados: {(ratio_borrados * 100):.2f}%")

Total de usuarios: 30572
Usuarios activos: 22544
Usuarios eliminados: 8028
Proporcion de usuarios borrados: 26.26%


Con la información obtenida no podemos determinar la causa por la cual la cuenta ha sido desactivada. 

In [22]:
kast_top_users = len(tweets_df[tweets_df.candidate_support == "Kast"].author_id.unique())
print("Usuarios que apoyan a Kast:", kast_top_users)

kast_active_users = len(tweets_df[(tweets_df.candidate_support == "Kast") & (tweets_df.status == "active")].author_id.unique())
print(f"Usuarios que apoyan a Kast activos: {kast_active_users} equivalente al {(kast_active_users / kast_top_users)*100:.2f}% del total de usuarios")

kast_deleted_accounts = len(tweets_df[(tweets_df.candidate_support == "Kast") & (tweets_df.status == "deleted")].author_id.unique())
print(f"Usuarios que apoyan a Kast eliminados: {kast_deleted_accounts} equivalente al {(kast_deleted_accounts / kast_top_users) * 100:.2f}% del total de usuarios")

Usuarios que apoyan a Kast: 16722
Usuarios que apoyan a Kast activos: 11705 equivalente al 70.00% del total de usuarios
Usuarios que apoyan a Kast eliminados: 5017 equivalente al 30.00% del total de usuarios


In [23]:
boric_top_users = len(tweets_df[tweets_df.candidate_support == "Boric"].author_id.unique())
print("Usuarios que apoyan a Boric:", boric_top_users)

boric_active_users = len(tweets_df[(tweets_df.candidate_support == "Boric") & (tweets_df.status == "active")].author_id.unique())
print(f"Usuarios que apoyan a Boric activos: {boric_active_users} equivalente al {(boric_active_users / boric_top_users)*100:.2f}% del total de usuarios")

boric_deleted_accounts = len(tweets_df[(tweets_df.candidate_support == "Boric") & (tweets_df.status == "deleted")].author_id.unique())
print(f"Usuarios que apoyan a Boric eliminados: {boric_deleted_accounts} equivalente al {(boric_deleted_accounts / boric_top_users) * 100:.2f}% del total de usuarios")

Usuarios que apoyan a Boric: 15611
Usuarios que apoyan a Boric activos: 12015 equivalente al 76.96% del total de usuarios
Usuarios que apoyan a Boric eliminados: 3596 equivalente al 23.04% del total de usuarios


Las cuentas eliminadas que apoyan a Kast constituyen el 30
% del total. En el caso de Boric, un 23.04% de las cuentas fueron eliminadas. Una hipótesis es que estas cuentas han sido cerradas por usar leguaje extremadamente violento, lo cual ocurriría en el caso de ambas candidaturas.

## Bots

Nuestro objetivo principal es identificar la magnitud de la presencia de Bots en el discurso publico referente al proceso eleccionario. Para clasificar una cuenta como Bot usamos un modelo llamado [Botometer](https://botometer.osome.iu.edu/) desarrollado por el [Observatory on Social Media](https://osome.iu.edu/) de la Indiana University. Elegimos ese modelo por su extensa [documentación](https://arxiv.org/abs/1602.00975) y validación pública.

El modelo toma una cuenta de usuario de Twitter y analiza su actividad histórica para determinar qué tan probable es que esa cuenta sea un Bot. Cuando el modelo le otorga una probabilidad a una cuenta de ser Bot mayor al 50% nosotros lo clasificamos como tal.

In [24]:
kast_bot_accounts = len(tweets_df[(tweets_df.candidate_support == "Kast") & (tweets_df.universal > 0.5)].author_id.unique())
print(f"Bots que apoyan a Kast: {kast_bot_accounts} equivalente al {(kast_bot_accounts / kast_active_users) * 100:.2f}% de los usuarios activos")
boric_bot_accounts = len(tweets_df[(tweets_df.candidate_support == "Boric") & (tweets_df.universal > 0.5)].author_id.unique())
print(f"Bots que apoyan a Boric: {boric_bot_accounts} equivalente al {(boric_bot_accounts / boric_active_users) * 100:.2f}% de los usuarios activos")

Bots que apoyan a Kast: 2762 equivalente al 23.60% de los usuarios activos
Bots que apoyan a Boric: 945 equivalente al 7.87% de los usuarios activos


## Bots por hashtag

In [25]:
# users per hashtag
hashtags_df = pd.DataFrame(columns=["hashtag", "active", "deleted", "bots"])

for hashtag in hashtags_list:
    active_users = 0
    deleted_users = 0
    bot_users = 0
    hashtag_df = tweets_df[tweets_df.text.str.contains(hashtag)]
    hashtag_users_list = list(hashtag_df.author_id.unique())
    for user in hashtag_users_list:
        users_info_df = users_df[users_df.id_str == user]
        if users_info_df.status.values[0] == "active":
            active_users += 1
        if users_info_df.status.values[0] == "deleted":
            deleted_users += 1
        if (users_info_df.universal > 0.5).values[0]:
            bot_users += 1
    hashtags_df = hashtags_df.append({
        "hashtag": hashtag,
        "active": active_users,
        "deleted": deleted_users,
        "bots": bot_users
    }, ignore_index=True)

hashtags_df["support"] = hashtags_df.hashtag.apply(lambda x: hashtags_support_dict[x])
hashtags_df

,hashtag,active,deleted,bots,support
0,#BoricPresidente,12731,3896,1152,Boric
1,#MujeresPorKast,4488,2040,1494,Kast
2,#TodoChileVotaKast,4497,2060,1549,Kast
3,#KastPresidente2022,4722,2102,1569,Kast
4,#TodosKast,3823,1760,1393,Kast
5,#BoricPresidente2022,3016,980,192,Boric
6,#BoricNoSeAtreveDebatir,2342,1078,834,Kast
7,#BoricNoSeLaPuede,2567,1145,988,Kast
8,#BoricMiente,3515,1558,1105,Kast
9,#KastPresidente,5512,2507,1746,Kast


In [26]:
def hashtag_stats(hashtag):
    hashtag_df = tweets_df[tweets_df.text.str.contains(hashtag)]
    tweets_count = len(hashtag_df)
    users_count = len(hashtag_df.author_id.unique())
    active_users_count = len(hashtag_df[hashtag_df.status == "active"])
    deleted_users_count = len(hashtag_df[hashtag_df.status == "deleted"])
    bot_users_count = len(hashtag_df[hashtag_df.universal > 0.5])

    active_users_ratio = active_users_count / tweets_count
    deleted_users_ratio = deleted_users_count / tweets_count
    bot_users_ratio = bot_users_count / active_users_count

    print("Hashtag:", hashtag)
    print("Tweets count:", tweets_count)
    print("Users count:", users_count)
    print("Active users:", active_users_count)
    print("Deeleted users:", deleted_users_count)
    print("Bot users count:", bot_users_count)
    print(f"Active users ratio: {(active_users_ratio * 100):.2f}%")
    print(f"Deleted users ratio: {(deleted_users_ratio * 100):.2f}%")
    print(f"Bot users ratio: {(bot_users_ratio * 100):.2f}%")

In [27]:
hashtag = "BoricPresidente"
hashtag_stats(hashtag)

Hashtag: BoricPresidente
Tweets count: 39536
Users count: 16628
Active users: 29648
Deeleted users: 9888
Bot users count: 2837
Active users ratio: 74.99%
Deleted users ratio: 25.01%
Bot users ratio: 9.57%


In [28]:
hashtag = "MujeresPorKast"
hashtag_stats(hashtag)

Hashtag: MujeresPorKast
Tweets count: 21458
Users count: 6529
Active users: 14320
Deeleted users: 7138
Bot users count: 6862
Active users ratio: 66.74%
Deleted users ratio: 33.26%
Bot users ratio: 47.92%


In [29]:
hashtag = "TodoChileVotaKast"
hashtag_stats(hashtag)

Hashtag: TodoChileVotaKast
Tweets count: 18632
Users count: 6557
Active users: 12263
Deeleted users: 6369
Bot users count: 5763
Active users ratio: 65.82%
Deleted users ratio: 34.18%
Bot users ratio: 47.00%


In [30]:
hashtag = "KastPresidente2022"
hashtag_stats(hashtag)

Hashtag: KastPresidente2022
Tweets count: 17711
Users count: 6829
Active users: 11927
Deeleted users: 5784
Bot users count: 5524
Active users ratio: 67.34%
Deleted users ratio: 32.66%
Bot users ratio: 46.32%


In [31]:
hashtag = "TodosKast"
hashtag_stats(hashtag)

Hashtag: TodosKast
Tweets count: 12353
Users count: 5583
Active users: 8069
Deeleted users: 4284
Bot users count: 3766
Active users ratio: 65.32%
Deleted users ratio: 34.68%
Bot users ratio: 46.67%


In [32]:
hashtag = "BoricPresidente2022"
hashtag_stats(hashtag)

Hashtag: BoricPresidente2022
Tweets count: 7615
Users count: 4002
Active users: 5688
Deeleted users: 1927
Bot users count: 398
Active users ratio: 74.69%
Deleted users ratio: 25.31%
Bot users ratio: 7.00%


In [33]:
hashtag = "BoricNoSeAtreveDebatir"
hashtag_stats(hashtag)

Hashtag: BoricNoSeAtreveDebatir
Tweets count: 7048
Users count: 3420
Active users: 4722
Deeleted users: 2326
Bot users count: 1969
Active users ratio: 67.00%
Deleted users ratio: 33.00%
Bot users ratio: 41.70%


In [34]:
hashtag = "BoricNoSeLaPuede"
hashtag_stats(hashtag)

Hashtag: BoricNoSeLaPuede
Tweets count: 5610
Users count: 3712
Active users: 3783
Deeleted users: 1827
Bot users count: 1574
Active users ratio: 67.43%
Deleted users ratio: 32.57%
Bot users ratio: 41.61%


In [35]:
hashtag = "BoricMiente"
hashtag_stats(hashtag)

Hashtag: BoricMiente
Tweets count: 9546
Users count: 5073
Active users: 6361
Deeleted users: 3185
Bot users count: 2237
Active users ratio: 66.64%
Deleted users ratio: 33.36%
Bot users ratio: 35.17%


In [36]:
hashtag = "KastPresidente"
hashtag_stats(hashtag)

Hashtag: KastPresidente
Tweets count: 22691
Users count: 8025
Active users: 15228
Deeleted users: 7463
Bot users count: 7019
Active users ratio: 67.11%
Deleted users ratio: 32.89%
Bot users ratio: 46.09%
